In [ ]:
import os

import numpy as np
import pandas as pd


import warnings

warnings.filterwarnings("ignore")

In [ ]:
data_folder = "/home/goubetcle/Documents/Pôle/Challenge 2022/data_challenge/data"
output_folder = "/home/goubetcle/Documents/Pôle/Challenge 2022/data_challenge/outputs"


In [ ]:
df_prev = pd.read_feather(os.path.join(data_folder, "df_prev_sans_obs2020.feather"))
df_prev = df_prev.loc[df_prev.echeance.isin([0.5,1,2,4])]
df_prev["hh_mm_cible"] = df_prev.date_cible.dt.hour + df_prev.date_cible.dt.minute / 60
df_prev["prev_obs_gap"] = df_prev.prev.values - df_prev.obs.values
df_prev.head(10)

In [ ]:
quantile_target = np.arange(0.005, 1, 0.005)
df_quantile_bygroup = df_prev[['type', 'echeance', 'hh_mm_cible', 'prev_obs_gap']].loc[df_prev.date_cible.dt.year < 2020].groupby(
    ['type', 'echeance', 'hh_mm_cible']).quantile(quantile_target).reset_index().rename(columns={"level_3":"quantile_niveau"})

df_quantile_bygroup.head()

In [ ]:
df_quantile_prev_4x199 = df_prev[['date_cible', 'date_lancement','type', 'echeance', 'hh_mm_cible', 'prev']].loc[df_prev.date_cible.dt.year == 2020]
n_examples = df_quantile_prev_4x199.shape[0]
df_quantile_prev_4x199 = pd.concat([df_quantile_prev_4x199] * len(quantile_target), ignore_index=True)
df_quantile_prev_4x199["quantile_niveau"] = np.repeat(quantile_target, n_examples)
df_quantile_prev_4x199 = df_quantile_prev_4x199.merge(df_quantile_bygroup, on=['type', 'echeance', 'hh_mm_cible', 'quantile_niveau'],
    how='left', sort='false')
df_quantile_prev_4x199["prev_q"] = df_quantile_prev_4x199.prev.values + df_quantile_prev_4x199.prev_obs_gap.values
df_quantile_prev_4x199 = df_quantile_prev_4x199[['date_cible', 'date_lancement', 'type', 'prev_q', 'quantile_niveau']]

In [ ]:
df_quantile_prev_4x199.tail(20)

In [ ]:
df_quantile_prev_Q1Q99 = df_quantile_prev_4x199.loc[df_quantile_prev_4x199.type=="consommation_residuelle"].loc[
    df_quantile_prev_4x199.quantile_niveau.isin([0.01, 0.99])].reset_index(drop=True)
df_quantile_prev_Q1Q99.head()

In [ ]:
df_quantile_prev_4x199.to_feather(os.path.join(output_folder, "demo_4x199.feather"), compression="zstd")
df_quantile_prev_Q1Q99.to_feather(os.path.join(output_folder, "demo_Q1Q99.feather"))